In [ ]:
from __future__ import print_function, unicode_literals
from builtins import open
from future import standard_library
standard_library.install_aliases()

import os
import nipype.interfaces.utility as util
import nipype.interfaces.ants as ants
import nipype.interfaces.io as io
from nipype.workflows.smri.ants import ANTSTemplateBuildSingleIterationWF
import nipype.pipeline.engine as pe 
from urllib import request, error, parse

# Study specific variables


In [ ]:
## NODES
# Data handling Nodes
datasource = pe.Node(interface=util.IdentityInterface(fields=['imageList', 'passiveImagesDictionariesList']),
                     run_without_submitting=True,
                     name='InputImages')
datasource.inputs.imageList = input_images
datasource.inputs.passiveImagesDictionariesList = input_passive_images
datasource.inputs.sort_filelist = True

datasink = pe.Node(io.DataSink(), name="datasink")
datasink.inputs.base_directory = os.path.join(requestedPath, "results")

# Template builder nodes

buildTemplateIteration1 = ANTSTemplateBuildSingleIterationWF('iteration01')

buildTemplateIteration2 = ANTSTemplateBuildSingleIterationWF('iteration02')

initAvg = pe.Node(interface=ants.AverageImages(), name='initAvg')
initAvg.inputs.dimension = 3
initAvg.inputs.normalize = True

In [ ]:
# build the workflow

tbuilder = pe.Workflow(name="ANTSTemplateBuilder")
tbuilder.base_dir = requestedPath

tbuilder.connect(datasource, "imageList", initAvg, "images")
tbuilder.connect(initAvg, 'output_average_image', buildTemplateIteration1, 'inputspec.fixed_image')
tbuilder.connect(datasource, 'imageList', buildTemplateIteration1, 'inputspec.images')
tbuilder.connect(datasource, 'passiveImagesDictionariesList', buildTemplateIteration1, 'inputspec.ListOfPassiveImagesDictionaries')
tbuilder.connect(buildTemplateIteration1, 'outputspec.template', buildTemplateIteration2, 'inputspec.fixed_image')
tbuilder.connect(datasource, 'imageList', buildTemplateIteration2, 'inputspec.images')
tbuilder.connect(datasource, 'passiveImagesDictionariesList', buildTemplateIteration2, 'inputspec.ListOfPassiveImagesDictionaries')
tbuilder.connect(buildTemplateIteration2, 'outputspec.template', datasink, 'PrimaryTemplate')
tbuilder.connect(buildTemplateIteration2, 'outputspec.passive_deformed_templates', datasink, 'PassiveTemplate')
tbuilder.connect(initAvg, 'output_average_image', datasink, 'PreRegisterAverage')

tbuilder.run()